## Reference: Continuous QAOA

The explanations and initial implementation in this notebook are based on the excellent blog post:  
**[Continuous QAOA — Luca Manelli (2019)](https://lucaman99.github.io/blog/2019/07/06/Continuous-QAOA.html)**.

In that work, the QAOA algorithm is implemented and simulated using local simulators, focusing on continuous-time evolution with a Hamiltonian approach.

---

## My Adaptation

I am adapting their approach to run on **Xanadu's backend simulator** — specifically, the **`simulon_gaussian`** backend. This involves small but important changes in how gates and circuits are constructed to be compatible with the photonic Gaussian model supported by Xanadu’s platform.

I will retain much of the core logic but adjust the encoding, cost Hamiltonian, and time-evolution implementation to suit the continuous-variable quantum framework used by `simulon_gaussian`.


#### Attempt to Run on Xanadu X8 Hardware

As part of this project, I attempted to implement the CV-QAOA algorithm on **Xanadu’s X8 hardware** — the only available photonic quantum chip on their cloud platform.

However, through discussions with Xanadu’s support team, I learned that the **X8 is a passive device** and **cannot implement displacement operations**, which are required for the cost unitary  
$$e^{i p \hat{x} / \hbar}.$$


##### Why It Cannot Be Implemented on X8

The unitary  
$$e^{i p \hat{x} / \hbar}$$  
**cannot be implemented** on the X8 chip because:

- X8 only supports **passive Gaussian operations** (unitaries on annihilation operators: $\hat{a} \mapsto U\hat{a}$).
- Passive operations **preserve the origin** in phase space (they can rotate or squeeze, but **not displace**).
- Displacements are **active transformations** — they change the **first moments** in phase space.
- Active operations like displacement require **external coherent drives** or **electro-optic modulation**, which **X8 does not support**.


##### Could It Be Emulated Indirectly?

A common question is whether clever combinations of passive components (e.g., interferometers, beam splitters, or phase shifters) could fake a displacement.

**The answer is no**:

- A displacement **cannot** be emulated with interferometers, beam splitters, or phase shifters alone.
- Even with **multiple modes**, there's no way to generate a displacement without **injecting an external coherent state or ancilla** — which X8 does not permit.



#### Transition to `simulon_gaussian` Backend and the `Pgate` Issue

After realizing that Xanadu’s **X8 hardware** does not support displacement operations due to its passive nature, I decided to move forward with the **`simulon_gaussian`** backend — a **simulator** that supports a broader set of Gaussian operations.

However, a new limitation emerged:  
Although `simulon_gaussian` supports **displacement and squeezing**, it **does not support the `Pgate`**, which is essential for implementing the quadratic phase  
$$\hat{P}(s) = \exp\left( i \frac{s}{2\hbar} \hat{x}^2 \right).$$

---

#####  Workaround: Emulating `Pgate` Using Conjugation

To overcome this, I used a known decomposition based on **conjugation with rotation gates**. Specifically, we use the identity:

$$
\hat{P}(s) = \hat{R}(-\pi/2) \, \hat{Z}(s) \, \hat{R}(\pi/2)
$$

Here’s what this means:

- **`Pgate(s)`** applies a momentum shear and is equivalent to a **quadratic phase in position**.
- **`Zgate(z)`** performs a **momentum displacement**:  
  $$\hat{Z}(z) = \exp(i z \hat{x})$$
- **`Rgate(θ)`** rotates the phase space:  
  $$\hat{R}(θ) = \exp(i θ \hat{n})$$  
  and acts on quadratures as:
  $$
  \begin{aligned}
  \hat{a} &\rightarrow \hat{a} \cos θ + \hat{a}^\dagger \sin θ \\
  \hat{p} &\rightarrow \hat{p} \cos θ - \hat{x} \sin θ
  \end{aligned}
  $$

While neither `Rgate` nor `Zgate` alone can replicate a `Pgate`, their **conjugation** can.

---

#####  Code Implementation

```python
Rgate(-np.pi/2) | q[0]
Zgate(-2 * hbar * p) | q[0]
Rgate(np.pi/2) | q[0]


In [1]:
import strawberryfields as sf
from strawberryfields.ops import *
# from strawberryfields.utils import scale
from numpy import pi, sqrt
import math
import random
import numpy as np

from matplotlib import pyplot as plt


In [2]:
prog = sf.Program(1)
eng = sf.RemoteEngine("simulon_gaussian")


iterations = 10
shots =  10
parabolic_min = 2
possible_parameters = [0.1, 0.3, 0.5, 0.7, 0.9, 1.1]
testing_trials = 100
optimal_value = math.inf
simulation = []



In [3]:
#Calculating loss

def function_optimize(x, parabolic_min):
    y = (x - parabolic_min)**2
    return y



In [4]:
def run_circuit(alpha_param, beta_param, parabolic_min):
    prog = sf.Program(1)

    with prog.context as q:
        for i in range(len(alpha_param)):
            Zgate(parabolic_min * 4 * beta_param[i]) | q[0]
            # Approximate Pgate(-4 * beta_param[i]) using R-Z-R sandwich (not equivalent)
            Rgate(-np.pi/2)     | q[0]
            Zgate(-4 * beta_param[i]) | q[0]
            Rgate(np.pi/2)      | q[0]
            Rgate(-1 * alpha_param[i]) | q[0]
        
        MeasureX | q[0]

    job = eng.run(prog, shots=1)
    return job

In [5]:
#Search for the optimal value
param_length = 5
op = []

alpha_param = []
beta_param = []


for i in range(param_length):
    alpha_param.append(random.choice(possible_parameters))
    beta_param.append(random.choice(possible_parameters))


for i in range(0, iterations):
    result = 0
    grid = []
    for h in range(0, shots):
        hello = run_circuit(alpha_param, beta_param, parabolic_min)
        result = result + hello.samples[0][0]
        grid.append(hello)
    result = result/shots
    calculation = function_optimize(result, parabolic_min)

    yer_a = alpha_param
    yer_b = beta_param

    if (calculation < optimal_value):
        simulation = grid
        optimal_value = calculation
        the_x_measurement = result
        op = [alpha_param[:], beta_param[:]]

    for i in range(param_length):
        alpha_param[i] = random.choice(possible_parameters)
        beta_param[i] = random.choice(possible_parameters)


optimal_parameters = op



2025-07-08 00:54:58,031 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-08 00:55:02,258 - INFO - The remote job 06e9337e-0be3-41a1-87b7-4dc2cd35a7f2 has been completed.
2025-07-08 00:55:02,579 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-08 00:55:06,786 - INFO - The remote job 94f6807d-34cb-4047-a527-2d83dfb519f7 has been completed.
2025-07-08 00:55:07,085 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-08 00:55:09,992 - INFO - The remote job 5641ff08-befb-4486-b71e-3878b77ad08d has been completed.
2025-07-08 00:55:10,289 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-08 00:55:13,079 - INFO - The remote job aec23dff-2efa-40a2-aee4-e9fa6835954d has been completed.
2025-07-08 00:55:13,409 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-08 00:55:17,847 - INFO - The remote job 15aeaf04-7fd

In [6]:
x_arr = range(0, testing_trials)
y_arr = []

for i in range(0, testing_trials):

    state = run_circuit(optimal_parameters[0], optimal_parameters[1], parabolic_min)
    y_arr.append(state.samples[0][0])

print(y_arr)

2025-07-08 01:01:57,029 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-08 01:02:01,153 - INFO - The remote job 3e711aaf-0265-403a-83d6-9e192cc67c82 has been completed.
2025-07-08 01:02:01,471 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-08 01:02:05,369 - INFO - The remote job 13dde5e8-fe54-4c03-bb85-4cca1285ce3f has been completed.
2025-07-08 01:02:05,664 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-08 01:02:10,021 - INFO - The remote job 12ee66f4-c431-4cd9-8c8a-2a49d3bff22f has been completed.
2025-07-08 01:02:10,369 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-08 01:02:14,521 - INFO - The remote job b37371ce-b2e0-4987-93e6-1e603e4f297c has been completed.
2025-07-08 01:02:14,811 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-08 01:02:19,133 - INFO - The remote job b09ed5e0-e51

[np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float64(2.86615359284236), np.float6